In [1]:
# Import libraries
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
from termcolor import colored
import pandas as pd
import numpy as np
import requests
import PyPDF2
import re
import plotly.graph_objects as go
import nltk
nltk.download('punkt')
from googletrans import Translator
import os
#import slate3k as slate
#!pip install slate3k

[nltk_data] Downloading package punkt to /home/jav/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
path_to_folder = './CV/'
translator = Translator()     #Funcion para traducir al español

def translate_text(text,src,dest):
    if text == "" or text is None:
        pass
    else:
        return translator.translate(text, src=src, dest=dest).text

In [3]:
jd = """
Acelera el progreso de tu carrera ahora mismo en InnovativeDev Global

¡Oportunidades REMOTAS en Desarrollo de Software!

Únete a una empresa líder en diseño y desarrollo de software en Latinoamérica, y forma parte de nuestra expansión global. Construimos soluciones para: banca, salud, educación y startups.

Nos encontramos en la búsqueda de Desarrolladores Python Ssr/Sr para sumarse a nuestro equipo.

¿Qué esperamos de vos?
• Graduado/a o estudiante avanzado/a de carreras de sistemas o similares.
• Experiencia mínima de 2 años con Python (Django, FastAPI, Flask)
• Experiencia mínima de 4 años en posiciones similares.

Además, valoramos
• Conocimientos previos en otras tecnologías.
• Experiencia con metodologías ágiles.
• Github, bitbucket, gitlab, etc.
• Experiencia con uso de herramientas de integración continua.

¿Por qué unirte al equipo?

Cultura de trabajo en equipo: One Team, One Goal

Ambiente dinámico y de alto rendimiento

Metas y objetivos claros

Desarrollo profesional y capacitación en tendencias tecnológicas

Oportunidades para demostrar tu talento

Compromiso con la diversidad, inclusión y no discriminación

Beneficios:

Trabajo remoto

3 semanas de vacaciones

Cobertura médica de primer nivel

Equilibrio entre vida laboral y personal con flexibilidad de horarios

Trabaja desde cualquier parte del mundo

Clases de Idiomas y Capacitaciones

¡Y mucho beneficios más!

Acerca de InnovativeDev Global:

Somos expertos en el desarrollo de soluciones digitales a medida, ayudando a nuestros clientes a alcanzar su máximo potencial. En InnovativeDev Global, entendemos que el mundo está en constante cambio, pero confiamos en nuestras relaciones basadas en el respeto y la confianza para enfrentar el futuro juntos.

Desplegamos tecnología. Construimos relaciones
"""

In [4]:
jd_en=translate_text(jd,'es','en')

In [5]:
def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()
    
    # Remove punctuation from the text
    text = re.sub('[^a-z]', ' ', text)
    
    # Remove numerical values from the text
    text = re.sub(r'\d+', '', text)
    
    # Remove extra whitespaces
    text = ' '.join(text.split())
    
    return text

In [6]:
# Model evaluation
model = Doc2Vec.load('cv_job_maching.model')

In [7]:
def evaluate_candidate(input_CV,input_JD):
    v1 = model.infer_vector(input_CV.split())
    v2 = model.infer_vector(input_JD.split())
    similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
    return round(similarity, 2)

In [19]:
all_files = os.listdir(path_to_folder)
df = pd.DataFrame(columns=['Filename', 'MatchValue'])
# Filtra solo los archivos PDF
pdf_files = [file for file in all_files if file.endswith('.pdf')]
j=0 #contador de archivos
for pdf_file in pdf_files:
    pdf_path = os.path.join(path_to_folder, pdf_file)
    
    with open(pdf_path, 'rb') as f:
        pdf = PyPDF2.PdfReader(f)
        resume_en = ''
        for i in range(len(pdf.pages)):
            pageObj = pdf.pages[i]
            text_page=pageObj.extract_text()
            text_to_translate=pageObj.extract_text()
            print("longitud " + str(len(text_to_translate)))
            if len(text_to_translate)<5000 and len(text_to_translate)>0:
                resume_en += translate_text(text_to_translate,'es','en')
            if len(text_to_translate)>5000:
                resume_en += translate_text(text_to_translate[:5000],'es','en')

    print("longitud" + str(len(resume)))
    #resume_en=translate_text(resume,'es','en')
    input_CV = preprocess_text(resume_en)
    input_JD = preprocess_text(jd_en)
    match=evaluate_candidate(input_CV,input_JD)
    print(match)
    df.loc[j] = {'Filename': pdf_file, 'MatchValue': match}
    j=j+1
    print("="*50)
   

longitud 2007
longitud0
60.67
longitud 1863
longitud 1646
longitud 285
longitud0
29.18
longitud 4412
longitud 4685
longitud0
42.76
longitud 1523
longitud 1225
longitud 17
longitud0
35.3
longitud 3114
longitud0
39.58
longitud 4195
longitud 186
longitud0
37.38
longitud 3365
longitud 3885
longitud 2243
longitud0
40.25
longitud 2546
longitud 2869
longitud 1050
longitud0
23.91
longitud 3029
longitud0
17.26
longitud 1313
longitud 925
longitud0
22.0
longitud 3311
longitud 2246
longitud0
32.74
longitud 5848
longitud 1982
longitud0
25.85
longitud 1512
longitud 2079
longitud 1717
longitud 1308
longitud 543
longitud 237
longitud 0
longitud0
45.64
longitud 3131
longitud 752
longitud0
26.83
longitud 1747
longitud 1986
longitud 2083
longitud 1425
longitud0
21.36
longitud 1402
longitud 1911
longitud 732
longitud0
-18.89
longitud 2283
longitud 2524
longitud 991
longitud0
17.51
longitud 3158
longitud 3014
longitud0
18.74
longitud 2614
longitud 3378
longitud 1983
longitud0
38.44
longitud 1917
longitud 2

In [24]:
df['MatchValue'] = df['MatchValue'].astype(float)
df_sorted = df.sort_values(by='MatchValue', ascending=False)
df_sorted.head(100)

,Filename,MatchValue
0,CV Efrain Lopez.pdf,60.67
39,Resume_Tomas_Garzon Duarte_RPT_21-09-13.pdf,52.15
58,Copia de CV Alvaro Omar Avaroa.pdf,51.24
79,CV Alvaro Omar Avaroa.pdf,50.58
63,CV Jonathan Larraguivel.pdf,49.63
...,...,...
29,CV Laura Codina Powers.pdf,5.81
28,CV Carlos Iván Pineda.pdf,4.67
75,Resume_Matias_Varea_AR_22-03-01.pdf,-7.15
60,CV Daniel Perez.pdf,-8.13
